# Using aphiaIDs to download OBIS occurrences for Bio/Eco EOVs

Created: 2024-09-13

The [IOOS Marine Life Data Network](https://ioos.github.io/marine_life_data_network/) has developed a list of aphiaIDs for taxa mentioned in the [GOOS Biology and Ecosystems Variables](https://goosocean.org/what-we-do/framework/essential-ocean-variables/). These lists are meant to assist in querying OBIS to evaluate the abundance and distribution of these taxa. Here is an R notebook of how this might work using mangroves as an example.

In [ ]:
library(gh)
library(readr)
library(robis)
library(dplyr)
library(htmlwidgets)

First, we will pull the file with the mangrove aphiaIDs from the Marine Life Data Network GitHub repo. **Note**: the acceptedTaxonIds in these files are based on what was up-to-date in the [WoRMS](https://marinespecies.org/) database as of the date this script was written.

In [5]:
mangroves <- read.csv("https://raw.githubusercontent.com/ioos/marine_life_data_network/main/eov_taxonomy/mangroves.csv")

Now we will do a bit of cleanup to get a list of aphiaIDs for mangroves so we can run our `robis` query using these as taxon identifiers.

In [3]:
mangroves$ID <- gsub("urn:lsid:marinespecies.org:taxname.", "", mangroves$acceptedTaxonId)
mangroves$ID <- as.numeric(mangroves$ID)
mangroveIdentifiers <- paste(mangroves$ID, collapse = ", ")

Using the taxonIDs from the last step, let's query OBIS for occurrence data for mangroves. This step may take a bit of time to run. When this script was written, there were over 130,000 records for mangroves in OBIS.

In [ ]:
mangrove_occ <- robis::occurrence(taxonid = mangroveIdentifiers)
# let's check how many occurrences we got from OBIS
nrow(mangrove_occ)

Now that we have all of our mangrove records from OBIS, we will map the global distribution of records using the `map_leaflet` function found in the `robis` package. With the leaflet functionality, you can zoom into records, click them, and it list the scientific name for that occurrence record.

In [ ]:
m <- map_leaflet(mangrove_occ,
            provider_tiles = "Esri.WorldGrayCanvas",
            popup = function(x) { x["scientificName"] },
            )
m

This next step is not required, but if you'd like to save this map to view it outside of R, here's how.

In [ ]:

saveWidget(m, "mangroveMap.html", selfcontained = TRUE)

For more information and code about how to get EOV data from OBIS, see the [NOAA GIS For The Ocean GitHub project](https://github.com/NOAA-GIS4Ocean/BioEco_EOV).